In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_user = 'user'
db_password = 'password'
db_host = 'localhost'  # 通常是 'localhost'
db_port = '23306'
db_name = 'database'

# 创建数据库引擎
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [3]:
predect_data = pd.read_csv('分析师预测指标文件.csv')
daily_data = pd.read_csv('日个股回报率文件.csv')

In [8]:
predect_data.tail()


,Stkcd,Rptdt,Fenddt,ReportID,Feps,Fpe
321693,873593,2023-09-12,2024-12-31,85798426,NaN,NaN
321694,873593,2023-09-12,2025-12-31,85798426,NaN,NaN
321695,873665,2023-12-19,2023-12-31,88058271,0.54,20.41
321696,873665,2023-12-19,2024-12-31,88058271,0.64,17.25
321697,873665,2023-12-19,2025-12-31,88058271,0.71,15.36


In [9]:
daily_data.head()

,Stkcd,Trddt,Dretwd,Dretnd
0,4,2022-01-04,0.029351,0.029351
1,4,2022-01-05,0.100050,0.100050
2,4,2022-01-06,-0.003183,-0.003183
3,4,2022-01-07,-0.036496,-0.036496
4,4,2022-01-10,0.017992,0.017992


In [6]:
table_name = 'predect_data' 
predect_data.to_sql(table_name, con=engine, if_exists='replace', index=False)
table_name = 'daily_data'
daily_data.to_sql(table_name, con=engine, if_exists='replace', index=False)

351708

In [10]:
# SQL 查询语句
query = """
SELECT 
    dd.Stkcd,          -- 股票代码
    dd.Dretwd,         -- 日回报率（含股息）
    dd.Dretnd,         -- 日回报率（不含股息）
    pd.Feps,           -- 预测每股收益
    pd.Fpe             -- 预测市盈率

FROM 
    daily_data dd
JOIN 
    predect_data pd ON pd.Stkcd = dd.Stkcd 
    AND pd.Rptdt = dd.Trddt                 

"""

df = pd.read_sql_query(query, engine)


In [11]:
df

,Stkcd,Dretwd,Dretnd,Feps,Fpe
0,9,-0.002988,-0.002988,2.70,19.00
1,9,-0.002988,-0.002988,2.12,25.00
2,9,-0.002988,-0.002988,1.75,30.00
3,30,-0.045704,-0.045704,0.86,6.00
4,30,-0.045704,-0.045704,0.73,8.00
...,...,...,...,...,...
14447,600885,-0.008075,-0.008075,1.79,18.00
14448,600885,-0.008075,-0.008075,1.50,21.50
14449,600893,-0.016791,-0.016791,1.09,38.62
14450,600893,-0.016791,-0.016791,0.84,50.35
